In [ ]:
import math
import random
random.seed(20)
size = 100 # 伤员总数
lambda1 = 1/6
lambda2 = 1/6


cha_list = []

type_A_casualty_num = [i+1 for i in range(size)] # A类伤员编号
type_B_casualty_num = [i+1 for i in range(size)] # B类伤员编号

inter_arrival_time_type_A = [random.expovariate(lambda1) for i in range(size)] # A类伤员到达区间
inter_arrival_time_type_B = [random.expovariate(lambda2) for i in range(size)] # B类伤员到达区间

arrival_time_type_A = [] # A类伤员到达时间
arrival_time_type_B = [] # B类伤员到达时间

arrival_time_type_A.append(inter_arrival_time_type_A[0]) # 第一个A类伤员
arrival_time_type_B.append(inter_arrival_time_type_B[0]) # 第一个B类伤员

# for i in range(size): # 初始化A类伤员到达时间
#   arrival_time_type_A.append(arrival_time_type_A[i-1]+inter_arrival_time_type_A[i]) # 上一个A类伤员的到达时间加上这一个A类伤员的到达区间

# for i in range(size): # 初始化B类伤员到达时间
#   arrival_time_type_B.append(arrival_time_type_B[i-1]+inter_arrival_time_type_B[i]) # 上一个B类伤员的到达时间加上这一个B类伤员的到达区间

for i in range(size): # 初始化A类伤员到达时间
  if arrival_time_type_A[i-1] <= 480.0:
    arrival_time_type_A.append(arrival_time_type_A[i-1]+inter_arrival_time_type_A[i]) # 上一个A类伤员的到达时间加上这一个A类伤员的到达区间

############################################################
for i in range(size): # 初始化B类伤员到达时间
  if arrival_time_type_A[i-1] <= 480.0:
    arrival_time_type_B.append(arrival_time_type_B[i-1]+inter_arrival_time_type_B[i]) # 上一个B类伤员的到达时间加上这一个B类伤员的到达区间


casualty_before_differ = [] # 未被分类的全部伤员 内容应为("类型",到达时间,编号)例如["A",0.827,1]

for i in range(size): # 将全部A类伤员加入未被分类的全部伤员
  try:
    casualty_before_differ.append(["A", arrival_time_type_A[i],type_A_casualty_num[i]]) # 按顺序加入B类伤员和他们的抵达时间和序号
  except:
    pass

for i in range(size): # 将全部B类伤员加入未被分类的全部伤员
  try:
    casualty_before_differ.append(["B", arrival_time_type_B[i],type_B_casualty_num[i]]) # 按顺序加入B类伤员和他们的抵达时间和序号
  except:
    pass

sorted_casualty_before_differ = sorted(casualty_before_differ, key=lambda x: x[1]) # 将未被分类的全部伤员安装到达时间进行排序

# heavy_medic_team_waiting = [] # 重伤医护组等待队列 内容应为("类型",编号)例如["A",1]
# light_medic_team_waiting = [] # 轻伤医护组等待队列 

sorted_casualty_before_differ = sorted_casualty_before_differ[:size]
#print(sorted_casualty_before_differ)
# print(heavy_medic_team_waiting) #  是历史遗留问题
# print(light_medic_team_waiting) # 这个也没看懂 同上

heavy_team_time = 0 # 重伤组完成上一个伤员的时间
light_team_time = 0 # 轻伤组完成上一个伤员的时间

# 重伤救治组急救１个Ａ类伤员平均需要３５ｍｉｎ，
# 而急救１个Ｂ类伤员平均需要４０ｍｉｎ；
# 医疗组急救１个Ｂ类伤员平均需要３０ｍｉｎ，
# 而急救１个Ａ类伤员平均需要４５ｍｉｎ。
for i in sorted_casualty_before_differ: # 判断伤病员类型并进行处理
  # print("heavy_team_time")
  # print(heavy_team_time)
  # print("light_team_time")
  # print(light_team_time)
  if i[0] == "A": # A类伤病员
    if heavy_team_time == 0:
      heavy_team_time = (i[1] + 35) # 重伤救治组急救１个Ａ类伤员平均需要３５ｍｉｎ
      i.append("heavy")
      i.append(heavy_team_time)
    elif heavy_team_time < i[1]:  # 如果重伤组空闲
      heavy_team_time = (heavy_team_time + 35) # 重伤救治组急救１个Ａ类伤员平均需要３５ｍｉｎ
      i.append("heavy")
      i.append(heavy_team_time)
    elif heavy_team_time - i[1]  <= 60: # 如果重伤组等待时间不超过一小时
      heavy_team_time = (heavy_team_time + 35) # 重伤救治组急救１个Ａ类伤员平均需要３５ｍｉｎ
      i.append("heavy")
      i.append(heavy_team_time)
    elif light_team_time < i[1]: # 如果医疗组有空闲
      light_team_time = (light_team_time + 45) # 医疗组急救１个Ａ类伤员平均需要４５ｍｉｎ
      i.append("light")
      i.append(light_team_time)
    elif light_team_time - i[1] > 120: # 医疗组没有空闲 并且等待时间超过两小时需要插队
      #TODO: 医疗组等待时间超过两小时排队或插队

      light_team_time = (light_team_time + 45) # 医疗组急救１个Ａ类伤员平均需要４５ｍｉｎ
      # i.append("cha")
      # i.append(light_team_time+45)
      # print("end cha")
      # cha_list.append(i)
      found = False
      for index in range(len(sorted_casualty_before_differ)): #为了更新需要插队前的所有时间
        if sorted_casualty_before_differ[index] is i: #停止计算因为到达了当前插队的人
          temp = i
          sorted_casualty_before_differ.remove(i)
          sorted_casualty_before_differ.insert(insert_here+1,temp)
          break
        elif sorted_casualty_before_differ[index][4]>i[1] and not found and sorted_casualty_before_differ[index][3] == "light" and sorted_casualty_before_differ[index+1][0] != "A": #找到正确的位置去插入，条件是最接近插入人的到达时间之后不为A类的地方
          start_here = sorted_casualty_before_differ[index][4]
          print("found")
          print( sorted_casualty_before_differ[index])
          insert_here = index
          found = True
        elif found and sorted_casualty_before_differ[index][3] == "light":#只更新light
          
          sorted_casualty_before_differ[index][4]+=45

      light_team_time = (light_team_time + 45) # 医疗组急救１个Ａ类伤员平均需要４５ｍｉｎ
      i.append("light")
      i.append(start_here+45)
    #   print("end cha")
    else:
      #直接排队进入医疗组不需要插队
      light_team_time = (light_team_time + 45) # 医疗组急救１个Ａ类伤员平均需要４５ｍｉｎ
      i.append("light")
      i.append(light_team_time)

  if i[0] == "B": # B类伤病员
    if light_team_time == 0:
      light_team_time = (i[1] + 30) # 医疗组急救１个B类伤员平均需要30ｍｉｎ
      i.append("light")
      i.append(light_team_time)
    elif light_team_time < i[1]: # 如果医疗组有空闲
      light_team_time = (light_team_time + 30) # 医疗组急救１个B类伤员平均需要30ｍｉｎ
      i.append("light")
      i.append(light_team_time)
    else:
      if heavy_team_time == 0:
        heavy_team_time = (i[1] + 40) # 重伤救治组急救１个B类伤员平均需要３５ｍｉｎ
        i.append("heavy")
        i.append(heavy_team_time)
      elif heavy_team_time < i[1]:  # 如果重伤组空闲
        heavy_team_time = (heavy_team_time + 40) # 重伤救治组急救１个B类伤员平均需要40ｍｉｎ
        i.append("heavy")
        i.append(heavy_team_time) 
      else: 
        light_team_time = (light_team_time + 30) # 医疗组急救１个B类伤员平均需要30ｍｉｎ
        i.append("light")
        i.append(light_team_time)



# TODO: 汇总评价指标结果

print(sorted_casualty_before_differ)





import csv
from datetime import datetime

now = datetime.now()
 
dt_string = now.strftime("%m月%d日__%H:%M:%S")
#dt_string = now.strftime("%M%D__%H:%M:%S")
file_name = ""
file_name += dt_string
file_name += ".csv"

with open(file_name, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["type", "arrival_time", "id","care_giver","leaving_time"])
    for c in sorted_casualty_before_differ:
      writer.writerow(c)


import pandas as pd
df = pd.read_csv(file_name)
print(df)


# for i in range(size): # 将伤员分类并加入对应的等待队列
#   try:
#     if sorted_casualty_before_differ[i][0] == "A": # 如果是A类伤员
#       heavy_medic_team_waiting.append(sorted_casualty_before_differ[i]) # 先加入重伤医护组等待队列 
#     elif sorted_casualty_before_differ[i][0] == "B": # 如果是B类伤员
#       light_medic_team_waiting.append(sorted_casualty_before_differ[i]) # 先加入轻伤医护组等待队列 
#   except:
#       pass

# def determine_hurt_type():
#   return

# def takeSecond(a):
#     return a[1]

# def main():

# #init
# possiable_need_heal[0].append("A") 
# possiable_need_heal[0].append(inter_arrival_time_type_A[0])
# arrival_time_type_A[0] = inter_arrival_time_type_A[0]

# for i in range(1,size):
#   arrival_time_type_A[i] = inter_arrival_time_type_A[i]+arrival_time_type_A[i-1]
#   possiable_need_heal[i].append("A")
#   possiable_need_heal[i].append(arrival_time_type_A[i])

# for i in range(0,size):
#   arrival_time_type_B[i] = inter_arrival_time_type_B[i]+arrival_time_type_B[i-1]
#   possiable_need_heal[i+size].append("B")
#   possiable_need_heal[i+size].append(arrival_time_type_B[i])

print(cha_list)

found
['B', 2.1522211390996766, 1, 'light', 32.15222113909968]
found
['A', 28.3276151116769, 2, 'light', 77.15222113909968]
found
['A', 35.217921320714, 5, 'light', 122.15222113909968]
found
['A', 37.05529217849079, 4, 'light', 167.15222113909968]
found
['A', 38.8605238238727, 6, 'light', 212.15222113909968]
found
['A', 41.27701285656786, 7, 'light', 257.15222113909965]
found
['A', 52.9803653175507, 8, 'light', 302.15222113909965]
found
['A', 53.617477345892176, 9, 'light', 347.15222113909965]
found
['A', 58.0853592826214, 10, 'light', 392.15222113909965]
found
['A', 61.26666452793001, 12, 'light', 437.15222113909965]
found
['A', 61.92051878295068, 14, 'light', 482.15222113909965]
found
['A', 79.89960721497066, 16, 'light', 527.1522211390997]
found
['A', 81.30371630559297, 18, 'light', 572.1522211390997]
found
['A', 85.26709645707015, 13, 'light', 617.1522211390997]
found
['A', 87.57348040038644, 15, 'light', 662.1522211390997]
found
['A', 93.44068784506422, 19, 'light', 707.1522211390